In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(torch.cuda.get_arch_list())

print(f"Is CUDA supported by this system?{torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
 
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device:{torch.cuda.current_device()}")
       
print(f"Name of current CUDA device:{torch.cuda.get_device_name(cuda_id)}")

In [ ]:
# print the names of the supported PYNQ boards
from finn.util.basic import pynq_part_map
print(pynq_part_map.keys())

In [ ]:
# change this if you have a different PYNQ board, see list above
pynq_board = "Pynq-Z2"
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 10

In [ ]:
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
model = ModelWrapper("onnx_models/mobilenetv2_w4a4_set_folding_factors.onnx")  # 
model = model.transform(ZynqBuild(platform = pynq_board, period_ns = target_clk_ns))

In [ ]:
from finn.transformation.fpgadataflow.make_pynq_driver import MakePYNQDriver
model = model.transform(MakePYNQDriver("zynq-iodma"))

In [ ]:
model.save("onnx_models/mobilenetv2_w4a4_post_synthesis.onnx")

In [ ]:
showInNetron("onnx_models/mobilenetv2_w4a4_post_synthesis.onnx")

In [ ]:
model = ModelWrapper("onnx_models/mobilenetv2_w4a4_post_synthesis.onnx")
sdp_node_middle = getCustomOp(model.graph.node[1])
postsynth_layers = sdp_node_middle.get_nodeattr("model")

showInNetron(postsynth_layers)

In [ ]:
model = ModelWrapper(postsynth_layers)
model.model.metadata_props

In [ ]:
model = ModelWrapper("onnx_models/mobilenetv2_w4a4_post_synthesis.onnx")
model.model.metadata_props

In [ ]:
! ls {model.get_metadata_prop("vivado_pynq_proj")}